# IMP example

#### Primer carreguem els mòduls de IMP. 

In [1]:
import os
import sys
import shutil
import random
import csv

import IMP.core
import IMP.pmi.dof
import IMP.pmi.macros
import IMP.pmi.restraints.basic
import IMP.pmi.restraints.stereochemistry
import IMP.pmi.tools
import IMP.pmi.topology
import IMP.pmi.output

In [2]:
print(sys.version)

3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]


In [3]:
print(IMP.__version__)

2.16.0


## Functions

In [4]:
def create_tag_molecules(pict_bdb, pict_components, pict_chains, state_tags):
    """
    Create IMP.pmi molecules using fluorophore coordinates
    :param pict_bdb:
    :param pict_components:
    :param pict_chains:
    :param state_tags:
    :return:
    """
    colors = ["blue", "orange", "yellow", "pink", "brown", "purple", "red", "green"]
    tag_molecules = list()
    for n in range(len(pict_components)):
        print('PMI: setting up tag', pict_components[n])
        molecule = state_tags.create_molecule(name=pict_components[n],
                                              sequence="",
                                              chain_id=pict_chains[n])

        atomic = molecule.add_structure(pdb_fn=pict_bdb,
                                        chain_id=pict_chains[n],
                                        res_range=[],  # add only specific set of residues
                                        soft_check=True,  # warns sequence mismatches between fasta and PDB sequence
                                        offset=0)  # to sync PDB residue numbering with FASTA numbering.

        molecule.add_representation(residues=atomic,  # adding all residues of PDB to representation
                                    bead_extra_breaks=[],  # Additional breakpoints for splitting beads.
                                    color=colors[n],
                                    bead_ca_centers=True,  # resolution=1 beads to be at CA centers
                                    resolutions=[1])

        tag_molecules.append(molecule)

    return tag_molecules


def get_pict_distance_restraints_dict(dr_tags_file):
    """
    Return dictionary of distances between tags and termini of cryo exocyst
    :param dr_tags_file:
    """
    # Defining PICT distances (from tags to termini of subunits)
    distances_to_tags_dict = dict()
    with open(dr_tags_file, "r") as dr:
        "dr_file --> tag,tag_residue,protein,protein_residue,raw_distance,Added distance,Final Distance"
        csv_reader = csv.reader(dr, delimiter=',')  # to parse the csv file delimited with commas.
        dr.readline()  # skip first line
        for line in csv_reader:
            tag = line[0]
            protein = line[2]
            tag_residue = int(line[1])
            protein_residue = int(line[3])
            chain_id = str(line[4])
            max_distance = float(line[7])
            raw_dist = float(line[5])
            sd = float(line[8])
            if raw_dist == 202:
                distances_to_tags_dict.setdefault("{}_frb-{}_C".format(tag, protein),
                                                  [tag, protein, tag_residue, protein_residue, chain_id, max_distance,
                                                   sd])

            elif raw_dist == 110:
                distances_to_tags_dict.setdefault("{}_gfp_c-{}_C".format(tag, protein),
                                                  [tag, protein, tag_residue, protein_residue, chain_id, max_distance,
                                                   sd])

            else:
                distances_to_tags_dict.setdefault("{}_gfp_n-{}_N".format(tag, protein),
                                                  [tag, protein, tag_residue, protein_residue, chain_id, max_distance,
                                                   sd])
    return distances_to_tags_dict

def set_pict_distance_restraints(distances_to_tags_dict, root_hierarchy, output_objects):
    """
    Setting and adding basic Harmonic Upper Bound distance restraints from distances_to_tags
    dictionary
    :return: distances_to_tags_dict, dr_list, output_objects
    """
    # Distance restraints to positioned tags:
    dr_list = list()
    for r, data in distances_to_tags_dict.items():
        tag = data[0]
        protein = data[1]
        tag_rn = int(data[2])
        protein_rn = int(data[3])
        max_distance = float(data[5])
        sd = float(data[6])
        kappa = IMP.core.Harmonic_get_k_from_standard_deviation(sd)
        dr = IMP.pmi.restraints.basic.DistanceRestraint(root_hier=root_hierarchy,
                                                        tuple_selection1=(tag_rn, tag_rn, tag),
                                                        tuple_selection2=(protein_rn, protein_rn, protein),
                                                        distancemin=0,
                                                        distancemax=max_distance,
                                                        # kappa=kappa,
                                                        label="restraint_{}".format(r))
        dr.add_to_model()
        dr.evaluate()
        dr_list.append(dr)
        output_objects.append(dr)
        print("\nSetting distance {}\n".format(dr.label))
        print("Max distance {}\n".format(max_distance))

    return distances_to_tags_dict, dr_list, output_objects


def distance_restraints_pict_cryo(dr_tags_file, root_hierarchy, output_objects):
    # Defining PICT distances (from tags to termini of subunits)
    distances_to_tags_dict = get_pict_distance_restraints_dict(dr_tags_file)

    # Distance restraints to positioned tags:
    return set_pict_distance_restraints(distances_to_tags_dict, root_hierarchy, output_objects)

## Define PATHS

define the peaths for the data (input) and where to store the results (output)

In [5]:
# ---------------------------
# 1. Define Input Data and Output Directories
# ---------------------------
data_directory = "/your/path/to/TFM/RESULTS/SEC3/IMP/input/data/"                             # input dir
# topology_file = data_directory + "SEC3_custom.topology"           # aquí defineixo paràmetres del model
topology_file = data_directory + "topology_wrb.txt" 
cryo_pdb_model = data_directory + "5yfp.pdb"                  # PDB cryoEM
pict_pdb_models = data_directory + "fluorophores_modified/"   # Fluorophores information
dr_tags_file = data_directory + "distance_restraints.csv"     # Restraints de distància en un CSV
output_directory = "./output/cryoEM/output/iterations/wr"    # Output dir
# output_index = sys.argv[1]  # N (number iterations )output prefix
output_index = 4

## Parameters

Define parameters for Monte Carlo Replica Exchange and for excluded volume and connectivity restraints.

In [6]:
# --------------------------
# 2. Scoring Parameters
# --------------------------

# --------------
# ----- Sterochemistry and Physical Restraints
ev_weight = 1.0  # Weight of excluded volume restraint
connectivity_scale = 1.2  # weight of Connectivity restraint

# --------------------
# 3. Sampling Parameters
# --------------------
num_frames = 100  # int(sys.argv[3])   # Number of frames in MC run
num_best_scoring_models = 3
num_mc_steps = 50  # Number of MC steps per frame
mc_temperature = 1.0  # Temperature for MC

# --- Simulated Annealing (sa)
#  - Alternates between two MC temperatures
sim_annealing = True  # If true, run simulated annealing
sa_min_temp_steps = 100  # Steps at min temp
sa_max_temp_steps = 20  # Steps at max temp
sa_temps = (1.0, 5.0)  # Sim annealing temperatures

# Replica Exchange (rex)
rex_temps = (1.0, 5.0)  # Temperature bounds for replica exchange


## MODELING 

First, define the model and load the data from the [IMP Topology file](https://integrativemodeling.org/2.15.0/doc/ref/classIMP_1_1pmi_1_1topology_1_1TopologyReader.html).



In [7]:
# Initialize model
m = IMP.Model()
# Read in the topology file --> to coarse model of exocyst
# Specify the directory where the PDB files and fasta files
topology = IMP.pmi.topology.TopologyReader(topology_file,
                                           pdb_dir="/home/gallegolab/Desktop/TFM/test_TFM/TFM/RESULTS/SEC3",
                                           fasta_dir="/home/gallegolab/Desktop/TFM/test_TFM/TFM/input_fasta")
                                           # pdb_dir=data_directory,
                                           # fasta_dir=data_directory)
# Use the BuildSystem macro to build states from the topology file
bs = IMP.pmi.macros.BuildSystem(m)

Define the fluorophores data, number of particles and their coordinates from the PDB files. we create a state inside the BuildSystem object defined as "bs"

In [8]:
# Define tag molecules (fluorophores) as chains and component names and load state
pict_pdb = random.choice(os.listdir(pict_pdb_models))  # get a tag model file randomly
pict_pdb_seqs = IMP.pmi.topology.PDBSequences(m, pict_pdb_models + pict_pdb)  # using IMP.model
pict_chains = [chain for chain in pict_pdb_seqs.sequences]
pict_components = ["tag_sec3", "tag_sec5", "tag_sec6", "tag_sec8", "tag_sec10", "tag_sec15", "tag_exo70", "tag_exo84"]
st_tags = bs.system.create_state()
# Each state can be specified by a topology file.
# Create tag molecules and load to system
tag_molecules = create_tag_molecules(pict_pdb_models + pict_pdb, pict_components, pict_chains, st_tags)

A
B
C
D
E
F
G
H
PMI: setting up tag tag_sec3
PMI: setting up tag tag_sec5
PMI: setting up tag tag_sec6
PMI: setting up tag tag_sec8


/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:560: StructureWarning: Substituting PDB residue type with FASTA residue type. Potentially dangerous.
  warnings.warn(


PMI: setting up tag tag_sec10
PMI: setting up tag tag_sec15
PMI: setting up tag tag_exo70
PMI: setting up tag tag_exo84


Load the structure from the Exocyst from the Topology File 

In [9]:
bs.add_state(topology)
system_molecules = bs.get_molecules()
cryo_components = [mol[1][0] for mol in system_molecules[1].items()]

BuildSystem.add_state: setting up molecule sec5 copy number 0
BuildSystem.add_state: molecule sec5 sequence has 971 residues
BuildSystem.add_state: ---- setting up domain 0 of molecule sec5
BuildSystem.add_state: -------- domain 0 of molecule sec5 extends from residue 1 to residue 154 
BuildSystem.add_state: -------- domain 0 of molecule sec5 represented by pdb file /home/gallegolab/Desktop/TFM/test_TFM/TFM/RESULTS/SEC3/IMP/input/data/5yfp.pdb 
BuildSystem.add_state: ---- setting up domain 1 of molecule sec5
BuildSystem.add_state: -------- domain 1 of molecule sec5 extends from residue 155 to residue 231 
BuildSystem.add_state: -------- domain 1 of molecule sec5 represented by pdb file /home/gallegolab/Desktop/TFM/test_TFM/TFM/RESULTS/SEC3/IMP/input/data/5yfp.pdb 
BuildSystem.add_state: ---- setting up domain 2 of molecule sec5
BuildSystem.add_state: -------- domain 2 of molecule sec5 extends from residue 232 to residue 971 
BuildSystem.add_state: -------- domain 2 of molecule sec5 rep

BuildSystem.add_state: ---- setting up domain 6 of molecule SEC3
BuildSystem.add_state: -------- domain 6 of molecule SEC3 extends from residue 1333 to residue 1336 
BuildSystem.add_state: -------- domain 6 of molecule SEC3 represented by BEADS 
BuildSystem.add_state: State 2 added


/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:1249: StructureWarning: Inconsistency between FASTA sequence and PDB sequence. FASTA type 74 "THR" and PDB type "MET"
  warnings.warn(


Finally, we create our system using a macro from IMP.

In [10]:
# Build the system representation and degrees of freedom
root_hierarchy, dof = bs.execute_macro(max_rb_trans=4.0,
                                       max_rb_rot=0.3,
                                       max_bead_trans=4.0,
                                       max_srb_trans=4.0,
                                       max_srb_rot=0.3)

/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:816: StructureWarning: Residues without representation in molecule tag_sec3: 1-3000
  warnings.warn(


BuildSystem.execute_macro: building representations
done building "tag_sec3" Chain A Copy: 0
done building "tag_sec5" Chain B Copy: 0
done building "tag_sec6" Chain C Copy: 0


/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:816: StructureWarning: Residues without representation in molecule tag_sec5: 1-3003
  warnings.warn(
/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:816: StructureWarning: Residues without representation in molecule tag_sec6: 1-3005
  warnings.warn(
/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:816: StructureWarning: Residues without representation in molecule tag_sec8: 1-3008
  warnings.warn(
/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:816: StructureWarning: Residues without representation in molecule tag_sec10: 1-3010
  warnings.warn(


done building "tag_sec8" Chain D Copy: 0
done building "tag_sec10" Chain E Copy: 0
done building "tag_sec15" Chain F Copy: 0
done building "tag_exo70" Chain G Copy: 0


/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:816: StructureWarning: Residues without representation in molecule tag_sec15: 1-3013
  warnings.warn(
/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:816: StructureWarning: Residues without representation in molecule tag_exo70: 1-3016
  warnings.warn(
/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:816: StructureWarning: Residues without representation in molecule tag_exo84: 1-3019
  warnings.warn(


...

done building "sec10" Chain D Copy: 0


/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:816: StructureWarning: Residues without representation in molecule sec15: 1-41, 187-188, 891-910
  warnings.warn(


done building "sec15" Chain E Copy: 0
done building "exo70" Chain F Copy: 0


/usr/lib/python3.8/dist-packages/IMP/pmi/topology/__init__.py:816: StructureWarning: Residues without representation in molecule exo84: 1-168
  warnings.warn(


done building "exo84" Chain G Copy: 0
done building "SEC3" Chain H Copy: 0
BuildSystem.execute_macro: setting up degrees of freedom
BuildSystem.execute_macro: -------- building rigid body ['sec5..0', 'sec5..1', 'sec5..2', 'SEC3..0', 'SEC3..1']
BuildSystem.execute_macro: -------- adding sec5..0
BuildSystem.execute_macro: -------- adding sec5..1
BuildSystem.execute_macro: -------- adding sec5..2
BuildSystem.execute_macro: -------- adding SEC3..0
BuildSystem.execute_macro: -------- adding SEC3..1
BuildSystem.execute_macro: -------- creating rigid body with max_trans 4.0 max_rot 0.3 non_rigid_max_trans 4.0
BuildSystem.execute_macro: -------- building rigid body ['sec6..0', 'sec6..1', 'sec6..2', 'SEC3..2', 'SEC3..3']
BuildSystem.execute_macro: -------- adding sec6..0
BuildSystem.execute_macro: -------- adding sec6..1
BuildSystem.execute_macro: -------- adding sec6..2
BuildSystem.execute_macro: -------- adding SEC3..2
BuildSystem.execute_macro: -------- adding SEC3..3
BuildSystem.execute_mac



We want only the exocyst to move, not the fluorophores, so we "disable movers" for the fluorophores.

Afther that, we set up a random initialization (shuffle_configuration)



In [11]:
# Fix tags rigid bodies but not the exocyst (the protein complex)
fixed_beads, fixed_rbs = dof.disable_movers(tag_molecules,
                                            [IMP.core.RigidBodyMover,
                                             IMP.pmi.TransformMover])

# Randomize the initial configuration before sampling, of only the molecules
# we are interested in (exocyst subunits)
IMP.pmi.tools.shuffle_configuration(root_hierarchy,
                                    excluded_rigid_bodies=dof.get_rigid_bodies(),
                                    max_translation=50,
                                    verbose=False,
                                    cutoff=5.0,
                                    niterations=100)

Fixing 0 movers
shuffling 4 rigid bodies
shuffling 126 flexible beads


### Define Scoring Functions Components

Now, we define the restraints

- Distance restraints: --> in distance-restraints.csv . We implement them with IMP Harmonic Upper Bound restraints.

- Connectivity --> to make sure each molecule is fully connected within itself.

- Excluded Volume --> to avoid clashes.

In [12]:
out_objects = list()  # reporter objects (for stat files)

### Distance Restraints

In [13]:
# Here we are defining a number of restraints on our system.
#  For all of them we call add_to_model() so they are incorporated into scoring
#  We also add them to the output_objects list, so they are reported in stat files

# Distances restraints from fluorophores to cryo subunits
dr_tags_dict, dr_list, output_objects = distance_restraints_pict_cryo(dr_tags_file, root_hierarchy, out_objects)


DistanceRestraint
Created distance restraint between Residue_3001 and Residue_1332

Setting distance restraint_tag_sec3_frb-SEC3_C

Max distance 215.95

DistanceRestraint
Created distance restraint between Residue_3002 and Residue_1332

Setting distance restraint_tag_sec3_gfp_c-SEC3_C

Max distance 124.0

DistanceRestraint
Created distance restraint between Residue_3003 and Residue_611

Setting distance restraint_tag_sec3_gfp_n-SEC3_N

Max distance 341.5

DistanceRestraint
Created distance restraint between Residue_3004 and Residue_971

Setting distance restraint_tag_sec5_gfp_c-sec5_C

Max distance 110.0

DistanceRestraint
Created distance restraint between Residue_3005 and Residue_1

Setting distance restraint_tag_sec5_gfp_n-sec5_N

Max distance 112.0

DistanceRestraint
Created distance restraint between Residue_3006 and Residue_805

Setting distance restraint_tag_sec6_frb-sec6_C

Max distance 201.95

DistanceRestraint
Created distance restraint between Residue_3007 and Residue_805

S

### Connectivity 

In [14]:
# Connectivity keeps things connected along the backbone (ignores if inside
# same rigid body)
all_molecules = IMP.pmi.tools.get_molecules(root_hierarchy)
cryo_molecules = [mol for mol in all_molecules if "tag" not in mol.get_name()]
cr_list = list()
for mol in cryo_molecules:
    mol_name = mol.get_name()
    IMP.pmi.tools.display_bonds(mol)
    cr = IMP.pmi.restraints.stereochemistry.ConnectivityRestraint(mol, scale=connectivity_scale)
    cr.add_to_model()
    cr.set_label(mol_name)
    output_objects.append(cr)
    cr_list.append(cr)

Adding sequence connectivity restraint between 1-10_bead  and  11-20_bead of distance 4.32
Adding sequence connectivity restraint between 11-20_bead  and  21-30_bead of distance 4.32
Adding sequence connectivity restraint between 21-30_bead  and  31-40_bead of distance 4.32
Adding sequence connectivity restraint between 31-40_bead  and  41-50_bead of distance 4.32
Adding sequence connectivity restraint between 41-50_bead  and  51-60_bead of distance 4.32
Adding sequence connectivity restraint between 51-60_bead  and  61-70_bead of distance 4.32
Adding sequence connectivity restraint between 61-70_bead  and  71-73_bead of distance 4.32
Adding sequence connectivity restraint between 71-73_bead  and  Residue_74 of distance 4.32
Adding sequence connectivity restraint between Residue_99  and  100-101_bead of distance 4.32
Adding sequence connectivity restraint between 100-101_bead  and  Residue_102 of distance 4.32
Adding sequence connectivity restraint between Residue_249  and  250-253_bea

Adding sequence connectivity restraint between Residue_32  and  33-42_bead of distance 4.32
Adding sequence connectivity restraint between 33-42_bead  and  43-52_bead of distance 4.32
Adding sequence connectivity restraint between 43-52_bead  and  53-62_bead of distance 4.32
Adding sequence connectivity restraint between 53-62_bead  and  63-64_bead of distance 4.32
Adding sequence connectivity restraint between 63-64_bead  and  Residue_65 of distance 4.32
Adding sequence connectivity restraint between Residue_331  and  332-341_bead of distance 4.32
Adding sequence connectivity restraint between 332-341_bead  and  342_bead of distance 4.32
Adding sequence connectivity restraint between 342_bead  and  Residue_343 of distance 4.32


### Excluded Volume

In [15]:
# Excluded Volume Restraint
#  To speed up this expensive restraint, we evaluate it at resolution 10
ev = IMP.pmi.restraints.stereochemistry.ExcludedVolumeSphere(included_objects=cryo_molecules,
                                                             resolution=10)
ev.set_weight(ev_weight)
ev.add_to_model()
output_objects.append(ev)

In [16]:
# Quickly move all flexible beads into place
dof.optimize_flexible_beads(nsteps=100)

optimize_flexible_beads: optimizing 104 flexible beads


## Sampling: Monte Carlo Replica Exchange

In [17]:
# --------------------------
# Monte-Carlo Sampling
# --------------------------

# This object defines all components to be sampled as well as the sampling protocol
mc1 = IMP.pmi.macros.ReplicaExchange0(m,
                                      root_hier=root_hierarchy,
                                      monte_carlo_sample_objects=dof.get_movers(),
                                      output_objects=output_objects,
                                      rmf_output_objects=output_objects,
                                      monte_carlo_temperature=mc_temperature,
                                      simulated_annealing=sim_annealing,
                                      simulated_annealing_minimum_temperature=min(sa_temps),
                                      simulated_annealing_maximum_temperature=max(sa_temps),
                                      simulated_annealing_minimum_temperature_nframes=sa_min_temp_steps,
                                      simulated_annealing_maximum_temperature_nframes=sa_max_temp_steps,
                                      replica_exchange_minimum_temperature=min(rex_temps),
                                      replica_exchange_maximum_temperature=max(rex_temps),
                                      number_of_best_scoring_models=num_best_scoring_models,
                                      monte_carlo_steps=num_mc_steps,
                                      number_of_frames=num_frames,
                                      global_output_directory=output_directory + "_" + str("SEC3"))

# Start Sampling
mc1.execute_macro()

Setting up MonteCarlo
Setting up ReplicaExchange
ReplicaExchange: Could not find MPI. Using Serial Replica Exchange
Setting up stat file
Stat info being written in the rmf file
Setting up replica stat file
Setting up best pdb files
Setting up and writing initial rmf coordinate file
got existing rex object
Setting up production rmf files
ReplicaExchange0: it generates initial.*.rmf3, stat.*.out, rmfs/*.rmf3 for each replica 
--- it stores the best scoring pdb models in pdbs/
--- the stat.*.out and rmfs/*.rmf3 are saved only at the lowest temperature
--- variables:
------ atomistic                      False
------ best_pdb_dir                   pdbs/
------ best_pdb_name_suffix           model
------ do_clean_first                 True
------ do_create_directories          True
------ geometries                     None
------ global_output_directory        ./output/cryoEM/output/iterations/wr_SEC3
------ initial_rmf_name_suffix        initial
------ molecular_dynamics_steps       10
--

## Output

IMP returns an output directory with:
- stats files --> files for analysis
- rmf --> file to see the iterations (open with Chimera)
- pdbs --> pdbs for each "state"
- initial.0.rmf3 --> Initial frame before the iterations.